## 1. 환경설정

In [1]:
import pandas as pd
from lightgbm import LGBMRegressor

## 2. 데이터 로드

In [2]:
df = pd.read_csv("data.csv", encoding='ansi')
df

,연번,연도,월,주차,공개일,조사기간(S),조사기간(F),긍정,부정,잘모름,...,국제%,북한_외교%,문화%,사회%,사건_사고%,정치%,스포츠%,지역%,"기타(날씨, 미분류)%",퍼센트합
0,1,2012,1,1,2012-01-06,2012-01-02,2012-01-06,26.0,63.0,11,...,9.953095,3.866834,13.980094,13.259352,8.763299,11.943714,6.475232,6.784121,5.880334,100
1,2,2012,1,2,2012-01-13,2013-01-09,2012-01-13,24.0,65.0,11,...,11.818475,3.075600,12.216367,13.237983,8.506291,12.345413,6.538337,6.645876,5.871599,100
2,3,2012,1,3,2012-01-20,2012-01-16,2012-01-20,22.0,65.0,13,...,12.098789,2.402460,12.118009,13.213531,8.591197,12.742648,6.409764,7.120892,6.515472,100
3,4,2012,1,4,2012-01-27,2012-01-25,2012-01-27,24.0,64.0,12,...,13.525856,3.102754,12.397582,13.472129,8.623237,10.100739,6.662189,6.675621,6.836803,100
4,5,2012,2,1,NaN,2012-01-30,2012-02-03,22.0,65.0,13,...,10.338562,2.247513,13.274675,13.771997,8.645754,11.027161,7.067712,7.096404,6.685157,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
571,572,2022,12,1,2022-12-02,2022-11-29,2022-12-01,31.0,60.0,9,...,8.591866,2.017361,8.278412,16.516637,8.061405,11.846970,11.605851,10.898569,5.312651,100
572,573,2022,12,2,2022-12-09,2022-12-06,2022-12-08,33.0,59.0,8,...,8.096226,2.472003,8.046454,14.914973,7.175446,12.700124,13.844878,11.455827,5.076732,100
573,574,2022,12,3,2022-12-16,2022-12-13,2022-12-15,36.0,56.0,8,...,8.890957,1.987987,8.857119,14.922595,7.275188,14.203536,7.122917,11.885627,5.997801,100
574,575,2022,12,4,연말 미조사,NaN,2022-12-22,NaN,NaN,100,...,9.525387,2.899031,8.846186,14.263232,6.891411,13.501201,5.764930,12.780585,6.477263,100


In [3]:
df.columns

Index(['연번', '연도', '월', '주차', '공개일', '조사기간(S)', '조사기간(F)', '긍정', '부정', '잘모름',
       '과학_IT', '경제', '국제', '북한_외교', '문화', '사회', '사건_사고', '정치', '스포츠', '지역',
       '기타(날씨, 미분류)', '주간합', '과학_IT%', '경제%', '국제%', '북한_외교%', '문화%', '사회%',
       '사건_사고%', '정치%', '스포츠%', '지역%', '기타(날씨, 미분류)%', '퍼센트합'],
      dtype='object')

### 2.1. 전처리

In [4]:
# 필요한 열만 추출

df_train = df.loc[:,['연도', '월', '주차', '긍정', '부정', '잘모름',
       '과학_IT', '경제', '국제', '북한_외교', '문화', '사회', '사건_사고', '정치', '스포츠', '지역',
       '기타(날씨, 미분류)', '주간합']]

df_train

,연도,월,주차,긍정,부정,잘모름,과학_IT,경제,국제,북한_외교,문화,사회,사건_사고,정치,스포츠,지역,"기타(날씨, 미분류)",주간합
0,2012,1,1,26.0,63.0,11,334,1335,870,338,1222,1159,766,1044,566,593,514,8741
1,2012,1,2,24.0,65.0,11,372,1464,1099,286,1136,1231,791,1148,608,618,546,9299
2,2012,1,3,22.0,65.0,13,455,1500,1259,250,1261,1375,894,1326,667,741,678,10406
3,2012,1,4,24.0,64.0,12,319,1066,1007,231,923,1003,642,752,496,497,509,7445
4,2012,2,1,22.0,65.0,13,447,1628,1081,235,1388,1440,904,1153,739,742,699,10456
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
571,2022,12,1,31.0,60.0,9,402,1697,1069,251,1030,2055,1003,1474,1444,1356,661,12442
572,2022,12,2,33.0,59.0,8,370,1585,976,298,970,1798,865,1531,1669,1381,612,12055
573,2022,12,3,36.0,56.0,8,444,1785,1051,235,1047,1764,860,1679,842,1405,709,11821
574,2022,12,4,NaN,NaN,100,393,1907,1150,350,1068,1722,832,1630,696,1543,782,12073


In [5]:
# 데이터 셋 체크

df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 576 entries, 0 to 575
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   연도           576 non-null    int64  
 1   월            576 non-null    int64  
 2   주차           576 non-null    int64  
 3   긍정           506 non-null    float64
 4   부정           506 non-null    float64
 5   잘모름          576 non-null    int64  
 6   과학_IT        576 non-null    int64  
 7   경제           576 non-null    int64  
 8   국제           576 non-null    int64  
 9   북한_외교        576 non-null    int64  
 10  문화           576 non-null    int64  
 11  사회           576 non-null    int64  
 12  사건_사고        576 non-null    int64  
 13  정치           576 non-null    int64  
 14  스포츠          576 non-null    int64  
 15  지역           576 non-null    int64  
 16  기타(날씨, 미분류)  576 non-null    int64  
 17  주간합          576 non-null    int64  
dtypes: float64(2), int64(16)
memory usage: 81.1 KB


약 70주치 데이터 손실

전체 데이터 576건 중 70주치 데이터 손실

570건 데이터 활용 가능

In [6]:
# null 값 채우기
df_clean = df_train.fillna(0)
df_clean.describe()

,연도,월,주차,긍정,부정,잘모름,과학_IT,경제,국제,북한_외교,문화,사회,사건_사고,정치,스포츠,지역,"기타(날씨, 미분류)",주간합
count,576.000000,576.000000,576.000000,576.000000,576.000000,576.000000,576.000000,576.000000,576.000000,576.000000,576.000000,576.000000,576.000000,576.000000,576.000000,576.000000,576.000000,576.000000
mean,2016.996528,6.557292,2.708333,38.788194,39.326389,21.885417,445.690972,1409.392361,1192.932292,472.348958,1389.206597,1442.324653,848.192708,1361.354167,626.131944,833.668403,544.822917,10566.065972
std,3.164475,3.463125,1.275181,20.207022,20.318192,29.387795,120.955809,329.927205,304.293349,278.507903,516.776494,452.420182,224.927774,461.863909,239.283452,351.748274,153.423098,1834.632962
min,2012.000000,1.000000,1.000000,0.000000,0.000000,3.000000,144.000000,459.000000,316.000000,105.000000,366.000000,283.000000,266.000000,459.000000,133.000000,180.000000,151.000000,2989.000000
25%,2014.000000,4.000000,2.000000,29.000000,25.750000,9.000000,353.000000,1154.250000,997.000000,286.000000,1071.250000,1104.750000,706.250000,1049.250000,472.750000,599.500000,438.250000,9140.500000
50%,2017.000000,7.000000,3.000000,41.000000,45.000000,10.000000,442.000000,1430.000000,1137.500000,397.500000,1207.500000,1336.000000,829.500000,1287.000000,564.000000,700.000000,538.000000,10731.500000
75%,2020.000000,10.000000,4.000000,49.000000,54.000000,15.000000,528.250000,1658.000000,1327.250000,585.500000,1485.750000,1703.500000,964.000000,1580.250000,745.000000,977.750000,623.250000,11916.500000
max,2022.000000,12.000000,5.000000,84.000000,93.000000,100.000000,913.000000,2402.000000,2958.000000,2896.000000,3753.000000,3402.000000,1550.000000,4171.000000,2304.000000,2293.000000,1343.000000,17425.000000


In [7]:
## 확인
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 576 entries, 0 to 575
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   연도           576 non-null    int64  
 1   월            576 non-null    int64  
 2   주차           576 non-null    int64  
 3   긍정           576 non-null    float64
 4   부정           576 non-null    float64
 5   잘모름          576 non-null    int64  
 6   과학_IT        576 non-null    int64  
 7   경제           576 non-null    int64  
 8   국제           576 non-null    int64  
 9   북한_외교        576 non-null    int64  
 10  문화           576 non-null    int64  
 11  사회           576 non-null    int64  
 12  사건_사고        576 non-null    int64  
 13  정치           576 non-null    int64  
 14  스포츠          576 non-null    int64  
 15  지역           576 non-null    int64  
 16  기타(날씨, 미분류)  576 non-null    int64  
 17  주간합          576 non-null    int64  
dtypes: float64(2), int64(16)
memory usage: 81.1 KB


In [8]:
## 학습용 데이터 셋 저장

df_clean.to_csv("data_clean.csv",encoding = 'cp949',index = False)

## 3. 모델링 진행

In [9]:
df = pd.read_csv("data_clean.csv",encoding = 'cp949')
df

,연도,월,주차,긍정,부정,잘모름,과학_IT,경제,국제,북한_외교,문화,사회,사건_사고,정치,스포츠,지역,"기타(날씨, 미분류)",주간합
0,2012,1,1,26.0,63.0,11,334,1335,870,338,1222,1159,766,1044,566,593,514,8741
1,2012,1,2,24.0,65.0,11,372,1464,1099,286,1136,1231,791,1148,608,618,546,9299
2,2012,1,3,22.0,65.0,13,455,1500,1259,250,1261,1375,894,1326,667,741,678,10406
3,2012,1,4,24.0,64.0,12,319,1066,1007,231,923,1003,642,752,496,497,509,7445
4,2012,2,1,22.0,65.0,13,447,1628,1081,235,1388,1440,904,1153,739,742,699,10456
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
571,2022,12,1,31.0,60.0,9,402,1697,1069,251,1030,2055,1003,1474,1444,1356,661,12442
572,2022,12,2,33.0,59.0,8,370,1585,976,298,970,1798,865,1531,1669,1381,612,12055
573,2022,12,3,36.0,56.0,8,444,1785,1051,235,1047,1764,860,1679,842,1405,709,11821
574,2022,12,4,0.0,0.0,100,393,1907,1150,350,1068,1722,832,1630,696,1543,782,12073


In [10]:
## 미조사 데이터 제외
df_clean = df.loc[df['긍정'] != 0]
df_clean

,연도,월,주차,긍정,부정,잘모름,과학_IT,경제,국제,북한_외교,문화,사회,사건_사고,정치,스포츠,지역,"기타(날씨, 미분류)",주간합
0,2012,1,1,26.0,63.0,11,334,1335,870,338,1222,1159,766,1044,566,593,514,8741
1,2012,1,2,24.0,65.0,11,372,1464,1099,286,1136,1231,791,1148,608,618,546,9299
2,2012,1,3,22.0,65.0,13,455,1500,1259,250,1261,1375,894,1326,667,741,678,10406
3,2012,1,4,24.0,64.0,12,319,1066,1007,231,923,1003,642,752,496,497,509,7445
4,2012,2,1,22.0,65.0,13,447,1628,1081,235,1388,1440,904,1153,739,742,699,10456
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
569,2022,11,3,29.0,61.0,10,438,1860,1210,689,1082,1927,1086,1488,745,1360,598,12483
570,2022,11,4,30.0,62.0,8,409,1768,1026,619,1000,1735,1021,1504,1194,1296,658,12230
571,2022,12,1,31.0,60.0,9,402,1697,1069,251,1030,2055,1003,1474,1444,1356,661,12442
572,2022,12,2,33.0,59.0,8,370,1585,976,298,970,1798,865,1531,1669,1381,612,12055


In [11]:
df_clean.columns

Index(['연도', '월', '주차', '긍정', '부정', '잘모름', '과학_IT', '경제', '국제', '북한_외교', '문화',
       '사회', '사건_사고', '정치', '스포츠', '지역', '기타(날씨, 미분류)', '주간합'],
      dtype='object')

In [12]:
# 학습용 데이터
X_train = df_clean.loc[df_clean['연도'] < 2022,['과학_IT', '경제', '국제', '북한_외교', '문화',
       '사회', '사건_사고', '정치', '스포츠', '지역', '기타(날씨, 미분류)', '주간합']]
y_train = df_clean.loc[df_clean['연도'] < 2022,['긍정']]

# y_train = df_clean.loc[df_clean['연도'] < 2022,['부정']]

# 테스트용 데이터
X_test = df_clean.loc[df_clean['연도'] == 2022,['과학_IT', '경제', '국제', '북한_외교', '문화',
       '사회', '사건_사고', '정치', '스포츠', '지역', '기타(날씨, 미분류)', '주간합']]
y_test = df_clean.loc[df_clean['연도'] == 2022,['긍정']]

# y_test = df_clean.loc[df_clean['연도'] == 2022,['부정']]

y_idx = df_clean.loc[df_clean['연도'] == 2022,['연도', '월', '주차']]

In [13]:
y_test

,긍정
524,41.0
525,42.0
526,41.0
527,42.0
529,41.0
530,40.0
531,43.0
532,45.0
533,43.0
534,42.0


In [14]:
X_train.columns = ["Science_IT","Ecnomic","Global","North_Korea","Culture","Social","Issue", 'Politic', 'Sports', 'Local', 'Etc', 'SUM']
y_train.columns = ['Target']

X_test.columns = ["Science_IT","Ecnomic","Global","North_Korea","Culture","Social","Issue", 'Politic', 'Sports', 'Local', 'Etc', 'SUM']
y_test.columns = ['Target']

In [15]:
## 모델링

lgbm = LGBMRegressor()
lgbm.fit(X_train,y_train)

lgbm.score(X_train,y_train)

0.9735583999719623

In [16]:
y_pred = lgbm.predict(X_test)

In [17]:
# 결과 해석

result = pd.DataFrame([y_pred,y_test.Target]).T
result.columns = ['예측',"실제값"]
result = pd.concat([y_idx.reset_index(drop=True),result.reset_index(drop=True)],axis = 1)
result

,연도,월,주차,예측,실제값
0,2022,1,1,46.512444,41.0
1,2022,1,2,37.303030,42.0
2,2022,1,3,54.039944,41.0
3,2022,1,4,33.872533,42.0
4,2022,2,2,33.833360,41.0
5,2022,2,3,37.385265,40.0
6,2022,2,4,34.765022,43.0
7,2022,3,1,47.232815,45.0
8,2022,3,2,26.539154,43.0
9,2022,3,3,43.776332,42.0


예측 성능이 좋다고는 못할 것 같다.

2022년 6월 1주차 예측값 33, 실제값 53

2022년 8월 1주차 예측값 40, 실제값 24

이런 것만 봐도,,,,

추가적인 데이터가 필요할 것으로 판단.

In [18]:
# 변수 중요도
feature_importance = pd.DataFrame([lgbm.feature_name_, lgbm.feature_importances_]).T
feature_importance.sort_values(by = 1, ascending = False)

,0,1
11,SUM,163
3,North_Korea,160
6,Issue,152
9,Local,149
10,Etc,148
7,Politic,144
8,Sports,144
5,Social,135
0,Science_IT,133
2,Global,124


변수 중요도를 확인했을때 주간 뉴스합계가 높은 비중을 차지하기는 한다.

그 다음으로 북한 외교 관련 뉴스

경제 뉴스가 가장 낮은 변수 중요도를 보이는 것은 의외의 결과,,,,